In [1]:
import timm
import torch
import torch.nn as nn
import sys
import os
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import datasets, transforms
sys.path.append(os.path.abspath(".."))
from data.ImageDataset import ImageDataset
from torchvision.models import efficientnet_b4, EfficientNet_B4_Weights

In [2]:

model = efficientnet_b4(weights=EfficientNet_B4_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 1)


In [3]:
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cuda


In [4]:
transform = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageDataset(
    annotations_path="/home/ec2-user/CS230Project/data/annotations/train.json",
    images_dir="/home/ec2-user/CS230Project/data/train",
    transform=transform,
)

val_dataset = ImageDataset(
    annotations_path="/home/ec2-user/CS230Project/data/annotations/val.json",
    images_dir="/home/ec2-user/CS230Project/data/val",
    transform=transform,
)


train_loader = DataLoader(train_dataset, batch_size=16, num_workers=7,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, num_workers=7, shuffle=False)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):

    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device).float()  


        outputs = model(images)
        outputs = outputs.view(-1)  
        labels = labels.view(-1)  
        loss = criterion(outputs, labels)

    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted = (outputs > 0.5).float()  
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")

    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device).float()

            outputs = model(images)
            outputs = outputs.view(-1)  
            labels = labels.view(-1)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100. * correct / total
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {val_accuracy:.2f}%")

    scheduler.step()

    checkpoint_path = f"/home/ec2-user/CS230Project/code/models/saved-weights/Efficientnet_b4/efficientnet_b4_{epoch+1}.pth"
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Model saved to {checkpoint_path}")



Training Epoch 1/10: 100%|██████████| 2757/2757 [16:27<00:00,  2.79it/s]


Epoch 1, Train Loss: 0.5495, Accuracy: 67.25%


Validation: 100%|██████████| 457/457 [01:04<00:00,  7.08it/s]


Validation Loss: 0.3181, Accuracy: 88.26%
Model saved to /home/ec2-user/CS230Project/code/models/saved-weights/Efficientnet_b4/efficientnet_b4_1.pth


Training Epoch 2/10: 100%|██████████| 2757/2757 [19:28<00:00,  2.36it/s]


...

Training Epoch 10/10: 100%|██████████| 2757/2757 [09:05<00:00,  5.05it/s]


Epoch 10, Train Loss: 0.0505, Accuracy: 97.90%


Validation: 100%|██████████| 457/457 [00:33<00:00, 13.60it/s]


Validation Loss: 0.5241, Accuracy: 91.34%
Model saved to /home/ec2-user/CS230Project/code/models/saved-weights/Efficientnet_b4/efficientnet_b4_10.pth


: 